In [1]:
# import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '1'


In [2]:
import os
import shutil
import pathlib
import gc
import pandas as pd
import ttach as tta
from pathlib import Path
from PIL import Image
import numpy as np
import cv2 as cv
import random
import matplotlib.pyplot as plt
# from tqdm.auto import tqdm
from tqdm.notebook import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import torch
from torch.utils.data import DataLoader, random_split
from torch.utils.data import Dataset

import torchvision
from torchvision import datasets
import cv2
from torch.cuda import amp

import torchvision.transforms as T
from torchvision.transforms import Compose, ToTensor, Resize
from torchvision.utils import make_grid
import optuna

import albumentations as A
from albumentations.pytorch import ToTensorV2
import segmentation_models_pytorch as smp
import timm

In [3]:
def seed_everything(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)

In [4]:
seed_everything(42)

In [5]:
CFGS = [ 
    
    {
        'model_name': 'Unet++',
        'backbone': 'tf_efficientnet_b7_ns',
        'img_size': [512, 512],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/contrail_01/output/nirjhar/qishentrialv2fpn512_tf_efficientnet_b7_ns_best_epochcv650lb675-00.bin',
        'threshold': 0.62, #0.24,
        'call_sign': "nir_01",
        'tta': True
    },

    {
        'model_name': 'Unet++',
        'backbone': 'tf_efficientnet_b7_ns',
        'img_size': [512, 512],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/contrail_01/output/nirjhar/qishentrialv512unetplus_tf_efficientnet_b7_ns_best_epochstage2cv669-00.bin',
        'threshold': 0.62, #0.24,
        'call_sign': "nir_02", 
        'tta': True
    },
    {
        'model_name': 'Unet',
        'backbone': 'eca_nfnet_l1',
        'img_size': [512, 512],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/contrail_01/output/nirjhar/unetplusecarnf01_eca_nfnet_l1_best_epochstage2cv656-00.bin',
        'threshold': 0.62, #0.24,
        'call_sign': "nir_03",
        'tta': True
    },
    {
    'model_name': 'Unet++',
        'backbone': 'tf_efficientnet_b8',
        'img_size': [512, 512],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/effb8modelv2fold0_tf_efficientnet_b8_best_epochstage2-00.bin',
        'threshold': 0.79,
        'call_sign': "nir_04", 
        'tta': True
    }, 
        

    {
        'model_name': 'Unet',
        'backbone': 'efficientnet-b7',
        'img_size': [256, 256],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01/Unet/efficientnet-b7-256/checkpoint_dice_fold3.pth',
        'threshold': 0.24, #0.24,
        'call_sign': "roh_01",
        'tta': True
    }, 
    {
        'model_name': 'Unet',
        'backbone': 'efficientnet-b7',
        'img_size': [256, 256],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_s2/Unet/efficientnet-b7-256/checkpoint_dice_fold0.pth',
        'threshold': 0.24, #0.24,
        'call_sign': "roh_02",
        'tta': True
    },     
    {
        'model_name': 'Unet',
        'backbone': 'efficientnet-b7',
        'img_size': [256, 256],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_s2/Unet/efficientnet-b7-256/checkpoint_dice_ctrl_fold3.pth',
        'threshold': 0.24, #0.24,
        'call_sign': "roh_03",
        'tta': True
    }, 
    
    
    {
        'model_name': 'Unet',
        'backbone': 'efficientnet-b7',
        'img_size': [256, 256],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_pl/Unet/efficientnet-b7-256/checkpoint_dice_ctrl_fold0.pth',
        'threshold': 0.24, #0.24,
        'call_sign': "roh_4",
        'tta': True
    }, 
    
    
    {
        'model_name': 'Unet',
        'backbone': 'efficientnet-b7',
        'img_size': [256, 256],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_pl_s2/Unet/efficientnet-b7-256/checkpoint_dice_fold3.pth',
        'threshold': 0.24, #0.24,
        'call_sign': "roh_5",
        'tta': True
    }, 
    
    #### above this are fixed
    
    
    {
        'model_name': 'Unet',
        'backbone': 'efficientnet-b7',
        'img_size': [256, 256],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_pl_687lb/Unet/efficientnet-b7-256/checkpoint_dice_ctrl_fold0.pth',
        'threshold': 0.24, #0.24,
        'call_sign': "roh_6",
        'tta': True
    },
    
    {
        'model_name': 'Unet',
        'backbone': 'efficientnet-b7',
        'img_size': [256, 256],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_pl_687lb/Unet/efficientnet-b7-256/checkpoint_dice_fold0.pth',
        'threshold': 0.24, #0.24,
        'call_sign': "roh_6",
        'tta': True
    },
    
    
    {
        'model_name': 'Unet',
        'backbone': 'efficientnet-b7',
        'img_size': [256, 256],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_pl_687lb/Unet/efficientnet-b7-256/checkpoint_dice_ctrl_fold3.pth',
        'threshold': 0.24, #0.24,
        'call_sign': "roh_6",
        'tta': True
    },
    
    {
        'model_name': 'Unet',
        'backbone': 'efficientnet-b7',
        'img_size': [256, 256],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_pl_687lb/Unet/efficientnet-b7-256/checkpoint_dice_fold3.pth',
        'threshold': 0.24, #0.24,
        'call_sign': "roh_6",
        'tta': True
    },
    
    
    
    
    {
        'model_name': 'Unet',
        'backbone': 'efficientnet-b7',
        'img_size': [256, 256],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_pl_687lb_s2/Unet/efficientnet-b7-256/checkpoint_dice_ctrl_fold0.pth',
        'threshold': 0.24, #0.24,
        'call_sign': "roh_6",
        'tta': True
    },
    
    {
        'model_name': 'Unet',
        'backbone': 'efficientnet-b7',
        'img_size': [256, 256],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_pl_687lb_s2/Unet/efficientnet-b7-256/checkpoint_dice_fold0.pth',
        'threshold': 0.24, #0.24,
        'call_sign': "roh_6",
        'tta': True
    },
    
    
    {
        'model_name': 'Unet',
        'backbone': 'efficientnet-b7',
        'img_size': [256, 256],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_pl_687lb_s2/Unet/efficientnet-b7-256/checkpoint_dice_ctrl_fold3.pth',
        'threshold': 0.24, #0.24,
        'call_sign': "roh_6",
        'tta': True
    },
    
    {
        'model_name': 'Unet',
        'backbone': 'efficientnet-b7',
        'img_size': [256, 256],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_pl_687lb_s2/Unet/efficientnet-b7-256/checkpoint_dice_fold3.pth',
        'threshold': 0.24, #0.24,
        'call_sign': "roh_6",
        'tta': True
    },
    
    
    
    
    {
        'model_name': 'Unet',
        'backbone': 'efficientnet-b7',
        'img_size': [384, 384],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_pl_687lb/Unet/efficientnet-b7-384/checkpoint_dice_ctrl_fold0.pth',
        'threshold': 0.24, #0.24,
        'call_sign': "roh_8",
        'tta': True
    },
    
    {
        'model_name': 'Unet',
        'backbone': 'efficientnet-b7',
        'img_size': [384, 384],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_pl_687lb/Unet/efficientnet-b7-384/checkpoint_dice_fold0.pth',
        'threshold': 0.24, #0.24,
        'call_sign': "roh_9",
        'tta': True
    },
    
    
    
    
]

ensemple_type = "No" #"No" #"mine"




In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [7]:
class ContrailDataset:
    def __init__(self, df, transform=None):
        self.df = df  
        self.images = df['image']
        self.labels = df['label']
        self.transform =transform
        
        
    def __len__(self):
        return len(self.images)
        
    def __getitem__(self, idx):
        image = np.load("../../input/" + self.images[idx]).astype(float)   
        label = np.load("../../input/" + self.labels[idx]).astype(float)
        
        
        # label_cls = 1 if label.sum() > 0 else 0
        if self.transform :
            data = self.transform(image=image, mask=label)
            image  = data['image']
            label  = data['mask']
            image = np.transpose(image, (2, 0, 1))
            label = np.transpose(label, (2, 0, 1))    
            
        return torch.tensor(image), torch.tensor(label)

In [8]:
import segmentation_models_pytorch as smp
import timm
n_blocks = 4

class TimmSegModel(nn.Module):
    def __init__(self, cfg, segtype='unet', pretrained=True):
        super(TimmSegModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 3, stride=1, padding=1, bias=False)
        self.conv2 = nn.Conv2d(6, 12, 3, stride=1, padding=1, bias=False)
        self.conv3 = nn.Conv2d(12, 36, 3, stride=1, padding=1, bias=False)
        self.mybn1 = nn.BatchNorm2d(6)
        self.mybn2 = nn.BatchNorm2d(12)
        self.mybn3 = nn.BatchNorm2d(36)     
        self.encoder = timm.create_model(
            cfg["backbone"],
            in_chans=3,
            features_only=True,
            drop_rate=0.8,
            drop_path_rate=0.5,
            pretrained=False
        )
        self.encoder.conv_stem=nn.Conv2d(6, 72, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)

        self.encoder.blocks[5] = nn.Identity()
        self.encoder.blocks[6] = nn.Sequential(
            nn.Conv2d(self.encoder.blocks[4][2].conv_pwl.out_channels, 320, 1),
            nn.BatchNorm2d(320),
            nn.ReLU6(),
        )
        tr = torch.randn(1,6,64,64)
        g = self.encoder(tr)
        encoder_channels = [1] + [_.shape[1] for _ in g]
        decoder_channels = [256, 128, 64, 32, 16]
        if segtype == 'unet':
            self.decoder = smp.decoders.unetplusplus.decoder.UnetPlusPlusDecoder(
                encoder_channels=encoder_channels[:n_blocks+1],
                decoder_channels=decoder_channels[:n_blocks],
                n_blocks=n_blocks,
            )

        self.segmentation_head = nn.Conv2d(decoder_channels[n_blocks-1], cfg['num_classes'], kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

    def forward(self,x):
        x = F.relu6(self.mybn1(self.conv1(x)))
        global_features = [0] + self.encoder(x)[:n_blocks]
        seg_features = self.decoder(*global_features)
        seg_features = self.segmentation_head(seg_features)
        return seg_features
    
    
    
def load_model_timmunetplusplus(cfg):
    model = TimmSegModel(cfg)
    model.load_state_dict(torch.load(cfg['model_pth']))
    return model

In [9]:
###TImmunetplusplus model Nirjhar

n_blocks = 4

class TimmSegModel(nn.Module):
    def __init__(self, backbone, cfg, segtype='unet', pretrained=True):
        super(TimmSegModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 3, stride=1, padding=1, bias=False)
        self.conv2 = nn.Conv2d(6, 12, 3, stride=1, padding=1, bias=False)
        self.conv3 = nn.Conv2d(12, 36, 3, stride=1, padding=1, bias=False)
        self.mybn1 = nn.BatchNorm2d(6)
        self.mybn2 = nn.BatchNorm2d(12)
        self.mybn3 = nn.BatchNorm2d(36)     
        self.encoder = timm.create_model(
            backbone,
            in_chans=3,
            features_only=True,
            drop_rate=0.8,
            drop_path_rate=0.5,
            pretrained=False
        )
        self.encoder.conv_stem=nn.Conv2d(6, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)

        self.encoder.blocks[5] = nn.Identity()
        self.encoder.blocks[6] = nn.Sequential(
            nn.Conv2d(self.encoder.blocks[4][2].conv_pwl.out_channels, 320, 1),
            nn.BatchNorm2d(320),
            nn.ReLU6(),
        )
        tr = torch.randn(1,6,64,64)
        g = self.encoder(tr)
        encoder_channels = [1] + [_.shape[1] for _ in g]
        decoder_channels = [256, 128, 64, 32, 16]
        if segtype == 'unet':
            self.decoder = smp.decoders.unetplusplus.decoder.UnetPlusPlusDecoder(
                encoder_channels=encoder_channels[:n_blocks+1],
                decoder_channels=decoder_channels[:n_blocks],
                n_blocks=n_blocks,
            )

        self.segmentation_head = nn.Conv2d(
            decoder_channels[n_blocks-1],
            cfg["num_classes"], 
            kernel_size=(3, 3),
            stride=(1, 1), 
            padding=(1, 1)
        )

    def forward(self,x):
        x = F.relu6(self.mybn1(self.conv1(x)))
        global_features = [0] + self.encoder(x)[:n_blocks]
        seg_features = self.decoder(*global_features)
        seg_features = self.segmentation_head(seg_features)
        return seg_features
    

#path='./exp/baselinev2/qishentrialv2_tf_efficientnet_b7_ns_last_epoch-00.bin'

def build_model_timmunetplus(backbone, cfg):
    model = TimmSegModel(backbone, cfg)
    return model

def load_model_timmunetplus(path, backbone, cfg):
    model = build_model_timmunetplus(backbone, cfg)
    model.load_state_dict(torch.load(path))
    return model


#### Model 2 Nirjhar

n_blocks =4
class TimmSegModel2(nn.Module):
    def __init__(self, backbone, segtype='unet', pretrained=True):
        super(TimmSegModel2, self).__init__()

        self.encoder = timm.create_model(
            backbone,
            in_chans=3,
            features_only=True,
            drop_rate=0.5,
            pretrained=False
        )
        g = self.encoder(torch.rand(1, 3, 128, 128))
        encoder_channels = [1] + [_.shape[1] for _ in g]
        decoder_channels = [256, 128, 64, 32, 16]
        if segtype == 'unet':
            self.decoder = smp.decoders.unetplusplus.decoder.UnetPlusPlusDecoder(
                encoder_channels=encoder_channels[:n_blocks+1],
                decoder_channels=decoder_channels[:n_blocks],
                n_blocks=n_blocks,
            )

        self.segmentation_head = nn.Sequential(
            nn.Conv2d(decoder_channels[n_blocks-1], 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.UpsamplingBilinear2d(scale_factor=1))

    def forward(self,x):
        global_features = [0] + self.encoder(x)[:n_blocks]
        seg_features = self.decoder(*global_features)
        seg_features = self.segmentation_head(seg_features)
        return seg_features
    


def build_model(backbone):
    model = TimmSegModel2(backbone, segtype='unet')
    return model

def load_model(path,backbone):
    model = build_model(backbone)
    model.load_state_dict(torch.load(path))
    return model


##################

n_blocks = 4

class TimmSegModel3(nn.Module):
    def __init__(self, cfg, segtype='unet', pretrained=True):
        super(TimmSegModel3, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 3, stride=1, padding=1, bias=False)
        self.conv2 = nn.Conv2d(6, 12, 3, stride=1, padding=1, bias=False)
        self.conv3 = nn.Conv2d(12, 36, 3, stride=1, padding=1, bias=False)
        self.mybn1 = nn.BatchNorm2d(6)
        self.mybn2 = nn.BatchNorm2d(12)
        self.mybn3 = nn.BatchNorm2d(36)     
        self.encoder = timm.create_model(
            cfg["backbone"],
            in_chans=3,
            features_only=True,
            drop_rate=0.8,
            drop_path_rate=0.5,
            pretrained=False
        )
        self.encoder.conv_stem=nn.Conv2d(6, 72, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)

        self.encoder.blocks[5] = nn.Identity()
        self.encoder.blocks[6] = nn.Sequential(
            nn.Conv2d(self.encoder.blocks[4][2].conv_pwl.out_channels, 320, 1),
            nn.BatchNorm2d(320),
            nn.ReLU6(),
        )
        tr = torch.randn(1,6,64,64)
        g = self.encoder(tr)
        encoder_channels = [1] + [_.shape[1] for _ in g]
        decoder_channels = [256, 128, 64, 32, 16]
        if segtype == 'unet':
            self.decoder = smp.decoders.unetplusplus.decoder.UnetPlusPlusDecoder(
                encoder_channels=encoder_channels[:n_blocks+1],
                decoder_channels=decoder_channels[:n_blocks],
                n_blocks=n_blocks,
            )

        self.segmentation_head = nn.Conv2d(decoder_channels[n_blocks-1], cfg['num_classes'], kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

    def forward(self,x):
        x = F.relu6(self.mybn1(self.conv1(x)))
        global_features = [0] + self.encoder(x)[:n_blocks]
        seg_features = self.decoder(*global_features)
        seg_features = self.segmentation_head(seg_features)
        return seg_features
    
    
    
def load_model3(cfg):
    model = TimmSegModel3(cfg)
    model.load_state_dict(torch.load(cfg['model_pth']))
    return model



## Rohit Model 
class Net(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg
        
        self.model = smp.Unet(
            encoder_name=cfg["backbone"],     
            encoder_weights=None,   
            in_channels=3,  
            classes=cfg["num_classes"],
            activation=None
        )
        
    
    def forward(self, inputs):
        mask = self.model(inputs)
        return mask

In [10]:
def dice_coef(y_true, y_pred, thr=0.5, epsilon=1e-6):
    y_true = y_true.to(torch.float32)
    y_pred = (y_pred>thr).to(torch.float32)
    inter = (y_true*y_pred).sum()
    den = y_true.sum() + y_pred.sum()
    dice = ((2*inter+epsilon)/(den+epsilon)).mean()
    
    return dice


def get_transform(img_size):
    transform = A.Compose([
        A.Resize(*img_size, interpolation=cv2.INTER_NEAREST),
    ], p=1.0)
    return transform

In [11]:
val_df =pd.read_csv("../../input/data_utils/val_df_filled.csv")

In [12]:
len(CFGS)

19

In [13]:
final_preds = []

for idx, cfg in enumerate(CFGS):   
#     if idx <= 8:
#         continue
    
    print(cfg)
    val_transform = get_transform(cfg['img_size'])
    valid_dataset = ContrailDataset(val_df, transform=val_transform)  

    valid_loader = DataLoader(
        valid_dataset, 
        batch_size = 32, #32, 
        shuffle = False, 
        num_workers = 2, 
        pin_memory = True, 
        drop_last = False
    )

    

    if ensemple_type == "mine":
        model = Net(cfg)    
        model = torch.nn.DataParallel(model).cuda()
        model.load_state_dict(torch.load(cfg['model_pth'], map_location=torch.device('cpu'))['model'])
    else:
        if idx <= 1:
            model = load_model_timmunetplus(cfg['model_pth'], cfg['backbone'], cfg)
        elif idx == 2:
            model = load_model(cfg['model_pth'], cfg['backbone'])
        elif idx == 3:
            model = load_model3(cfg)
        else:
            model = Net(cfg)    
            model = torch.nn.DataParallel(model).cuda()
            model.load_state_dict(torch.load(cfg['model_pth'], map_location=torch.device('cpu'))['model'])

    if cfg['tta']:
        if ensemple_type == "mine":
            model = tta.SegmentationTTAWrapper(model, tta.aliases.flip_transform(), merge_mode='mean')
        else:
            if idx <= 3:
                model = tta.SegmentationTTAWrapper(model, tta.aliases.hflip_transform(), merge_mode='mean')
            else:
                model = tta.SegmentationTTAWrapper(model, tta.aliases.flip_transform(), merge_mode='mean')

    
    
    model.to(device)
    model.eval()
    
    preds = []
    masks_ = []
    
    for index, (images, masks) in enumerate(tqdm(valid_loader)):  
        images  = images.to(device, dtype=torch.float)
        masks  = masks.to(device, dtype=torch.float)
        if cfg['img_size'][0] != 256:
            masks = torch.nn.functional.interpolate(masks, size=256, mode='nearest') 
            
        masks_.append(torch.squeeze(masks, dim=1))

        with torch.inference_mode():
            images = torch.nn.functional.interpolate(images, size=cfg['img_size'][0], mode='nearest')
            pred = model(images).sigmoid()              
            pred = torch.nn.functional.interpolate(pred, size=256, mode='nearest') 
            preds.append(torch.squeeze(pred, dim=1))
            
               
            
    model_masks = torch.cat(masks_, dim=0)
    model_preds = torch.cat(preds, dim=0)
        
    model_masks = torch.flatten(model_masks, start_dim=0, end_dim=1)
    model_preds = torch.flatten(model_preds, start_dim=0, end_dim=1)  
    
#     # save
#     torch.save(model_preds, f"../../output/final_preds/{cfg['call_sign']}.pt")    
#     if idx == 6:
#         torch.save(model_masks, f'../../output/final_preds/val_masks.pt')

    
    best_threshold = 0.0
    best_dice_score = 0.0
    for threshold in [i / 100 for i in range(101)] :
        score = dice_coef(model_masks, model_preds.sigmoid(), thr=threshold).cpu().detach().numpy() 
        if score > best_dice_score:
            best_dice_score = score
            best_threshold = threshold
    
        
    print(best_dice_score, best_threshold)
    final_preds.append(model_preds)
    
    del model
    torch.cuda.empty_cache()
    gc.collect()

{'model_name': 'Unet++', 'backbone': 'tf_efficientnet_b7_ns', 'img_size': [512, 512], 'num_classes': 1, 'model_pth': '/home/rohits/pv1/contrail_01/output/nirjhar/qishentrialv2fpn512_tf_efficientnet_b7_ns_best_epochcv650lb675-00.bin', 'threshold': 0.62, 'call_sign': 'nir_01', 'tta': True}


  0%|          | 0/58 [00:00<?, ?it/s]

0.65715754 0.58
{'model_name': 'Unet++', 'backbone': 'tf_efficientnet_b7_ns', 'img_size': [512, 512], 'num_classes': 1, 'model_pth': '/home/rohits/pv1/contrail_01/output/nirjhar/qishentrialv512unetplus_tf_efficientnet_b7_ns_best_epochstage2cv669-00.bin', 'threshold': 0.62, 'call_sign': 'nir_02', 'tta': True}


  0%|          | 0/58 [00:00<?, ?it/s]

0.6720833 0.51
{'model_name': 'Unet', 'backbone': 'eca_nfnet_l1', 'img_size': [512, 512], 'num_classes': 1, 'model_pth': '/home/rohits/pv1/contrail_01/output/nirjhar/unetplusecarnf01_eca_nfnet_l1_best_epochstage2cv656-00.bin', 'threshold': 0.62, 'call_sign': 'nir_03', 'tta': True}


...

0.67612785 0.52
{'model_name': 'Unet', 'backbone': 'efficientnet-b7', 'img_size': [384, 384], 'num_classes': 1, 'model_pth': '/home/rohits/pv1/Contrail_Detection/output/exp_01_pl_687lb/Unet/efficientnet-b7-384/checkpoint_dice_ctrl_fold0.pth', 'threshold': 0.24, 'call_sign': 'roh_8', 'tta': True}


  0%|          | 0/58 [00:00<?, ?it/s]

0.6749311 0.59
{'model_name': 'Unet', 'backbone': 'efficientnet-b7', 'img_size': [384, 384], 'num_classes': 1, 'model_pth': '/home/rohits/pv1/Contrail_Detection/output/exp_01_pl_687lb/Unet/efficientnet-b7-384/checkpoint_dice_fold0.pth', 'threshold': 0.24, 'call_sign': 'roh_9', 'tta': True}


  0%|          | 0/58 [00:00<?, ?it/s]

0.67299825 0.52


In [14]:
# ### Load presaved predictions
# call_signs = [
#     "nir_01", "nir_02", "nir_03", "nir_04", "roh_01", "roh_02", "roh_03", "roh_04", "roh_05",
# #     "roh_06" , "roh_07" , "roh_08","roh_09"
# ]

# saved_preds = []

# for sign in tqdm(call_signs):
#     saved_preds.append(torch.load(f"/home/rohits/pv1/Contrail_Detection/output/final_preds/{sign}.pt"))
    

In [15]:
model_masks = torch.load(f"/home/rohits/pv1/Contrail_Detection/output/final_preds/val_masks.pt")

In [17]:
if len(final_preds) > 0:
    saved_preds.extend(final_preds)
    final_preds = saved_preds
else:
    final_preds = saved_preds

In [16]:
# final_preds = saved_preds

len(final_preds)

19

In [17]:
# final_preds = saved_preds

# final_preds = final_preds[:8]

In [21]:
def boost_dice(params, TH=None):    
    
    for index, val in enumerate(params.keys()):
        if index == 0:            
            preds = params[val]*final_preds[0]
        else:
            preds += params[val]*final_preds[index]
    
    param_sum = 0
    for key, val in params.items():
        param_sum += val

    preds = preds/param_sum
    
    
    if TH:
        best_threshold = TH
#         preds1 = (preds.sigmoid()).double()
        preds1 = (preds).double()
        best_score = dice_coef(model_masks, preds1, thr=best_threshold).cpu().detach().numpy() # get_score(labels, preds) 
    else:
        best_threshold = 0.0
        best_score = 0.0
        for threshold in [i / 100 for i in range(101)] :    
#             preds1 = (nn.Sigmoid()(preds)>threshold).double()    
            preds1 = ((preds)>threshold).double()            

            score = dice_coef(model_masks, preds1, thr=threshold).cpu().detach().numpy() # get_score(labels, preds) 
            if score > best_score:
                best_score = score
                best_threshold = threshold
            
#     best_score = score
#     best_threshold = threshold
    return best_score, best_threshold

In [22]:
def objective(trial):
    TH = None
    
    params = {}    
    for i in range(len(final_preds)):
#         if i == 0:
#             params["w1"] = 0.0
#         elif i == 5:
#             params["w6"] = 0.0
            
#         elif i == 8:
#             params["w9"] = 0.0
            
#         elif i == 12:
#             params["w13"] = 0.0
        
#         else:
        params[f"w{i+1}"] = trial.suggest_float(f'w{i+1}', 0, 1) 
        
        
#     params['w1'] = 0.0
#     params['w6'] = 0.0
#     params['w9'] = 0.0
#     params['w13'] = 0.0
    
        
    score, best_threshold  = boost_dice(params, TH=TH)
    params['threshold'] = best_threshold
    if not TH:
        print(params)
    return score

study = optuna.create_study(direction='maximize')
# study = optuna.create_study(direction='minimize')

study.optimize(objective, n_trials=1500)

best_params = study.best_params
best_params

[I 2023-06-26 14:26:55,625] A new study created in memory with name: no-name-7c132f63-2378-4ade-90a5-f6182d2a48b2
[I 2023-06-26 14:26:56,565] Trial 0 finished with value: 0.6837315559387207 and parameters: {'w1': 0.26312178613940174, 'w2': 0.9450732859368834, 'w3': 0.08952266610811288, 'w4': 0.46682984884705014, 'w5': 0.4005836758192325, 'w6': 0.18414314405398424, 'w7': 0.07078890457498066, 'w8': 0.5997136012620734, 'w9': 0.7904893313954023, 'w10': 0.9035848072551979, 'w11': 0.5144698669808951, 'w12': 0.7701707807735837, 'w13': 0.6781542028260878, 'w14': 0.17075755660969816, 'w15': 0.4309261730684454, 'w16': 0.3725536188952141, 'w17': 0.1389694825773039, 'w18': 0.8340989799721528, 'w19': 0.9982848258853001}. Best is trial 0 with value: 0.6837315559387207.


{'w1': 0.26312178613940174, 'w2': 0.9450732859368834, 'w3': 0.08952266610811288, 'w4': 0.46682984884705014, 'w5': 0.4005836758192325, 'w6': 0.18414314405398424, 'w7': 0.07078890457498066, 'w8': 0.5997136012620734, 'w9': 0.7904893313954023, 'w10': 0.9035848072551979, 'w11': 0.5144698669808951, 'w12': 0.7701707807735837, 'w13': 0.6781542028260878, 'w14': 0.17075755660969816, 'w15': 0.4309261730684454, 'w16': 0.3725536188952141, 'w17': 0.1389694825773039, 'w18': 0.8340989799721528, 'w19': 0.9982848258853001, 'threshold': 0.29}


[I 2023-06-26 14:26:57,493] Trial 1 finished with value: 0.6840323805809021 and parameters: {'w1': 0.7316804856912187, 'w2': 0.8627632777477986, 'w3': 0.3677947734150645, 'w4': 0.3394190481573388, 'w5': 0.01870333000294122, 'w6': 0.5973800679227081, 'w7': 0.7268556364765152, 'w8': 0.7576276578312596, 'w9': 0.03350140006563118, 'w10': 0.7431213330286234, 'w11': 0.9739945341792932, 'w12': 0.735583218551668, 'w13': 0.7588869065551042, 'w14': 0.2063383887149366, 'w15': 0.2507174035108154, 'w16': 0.8942151487291833, 'w17': 0.828373994570258, 'w18': 0.17276268013289064, 'w19': 0.6811068339774633}. Best is trial 1 with value: 0.6840323805809021.


{'w1': 0.7316804856912187, 'w2': 0.8627632777477986, 'w3': 0.3677947734150645, 'w4': 0.3394190481573388, 'w5': 0.01870333000294122, 'w6': 0.5973800679227081, 'w7': 0.7268556364765152, 'w8': 0.7576276578312596, 'w9': 0.03350140006563118, 'w10': 0.7431213330286234, 'w11': 0.9739945341792932, 'w12': 0.735583218551668, 'w13': 0.7588869065551042, 'w14': 0.2063383887149366, 'w15': 0.2507174035108154, 'w16': 0.8942151487291833, 'w17': 0.828373994570258, 'w18': 0.17276268013289064, 'w19': 0.6811068339774633, 'threshold': 0.24}


[I 2023-06-26 14:26:58,421] Trial 2 finished with value: 0.6836480498313904 and parameters: {'w1': 0.8272994967601053, 'w2': 0.7772562699356311, 'w3': 0.03833793547010478, 'w4': 0.5023029004358731, 'w5': 0.6933844290892315, 'w6': 0.777525461203317, 'w7': 0.3629220909780333, 'w8': 0.7362709727798088, 'w9': 0.03535963604479975, 'w10': 0.5727110565566065, 'w11': 0.48501029856840616, 'w12': 0.5137692756559225, 'w13': 0.15554951522686022, 'w14': 0.001620882805160151, 'w15': 0.7599658632856139, 'w16': 0.7641776375338561, 'w17': 0.06019283110598839, 'w18': 0.05348447077191798, 'w19': 0.6436043858313583}. Best is trial 1 with value: 0.6840323805809021.


...

[I 2023-06-26 14:54:43,346] Trial 1498 finished with value: 0.6842154860496521 and parameters: {'w1': 0.7520930021204846, 'w2': 0.7729516005149567, 'w3': 0.4298062276302277, 'w4': 0.7927908874291064, 'w5': 0.11433162135152292, 'w6': 0.30282701231865444, 'w7': 0.3733731718071775, 'w8': 0.7940605526136804, 'w9': 0.5294075184998772, 'w10': 0.6334966975956078, 'w11': 0.07054232680872888, 'w12': 0.24441782663367248, 'w13': 0.149726210423115, 'w14': 0.9473609859428428, 'w15': 0.03757333665775856, 'w16': 0.5893612518007871, 'w17': 0.18256572084955733, 'w18': 0.9825047792505454, 'w19': 0.42999776667485945}. Best is trial 1014 with value: 0.6852377653121948.


{'w1': 0.7520930021204846, 'w2': 0.7729516005149567, 'w3': 0.4298062276302277, 'w4': 0.7927908874291064, 'w5': 0.11433162135152292, 'w6': 0.30282701231865444, 'w7': 0.3733731718071775, 'w8': 0.7940605526136804, 'w9': 0.5294075184998772, 'w10': 0.6334966975956078, 'w11': 0.07054232680872888, 'w12': 0.24441782663367248, 'w13': 0.149726210423115, 'w14': 0.9473609859428428, 'w15': 0.03757333665775856, 'w16': 0.5893612518007871, 'w17': 0.18256572084955733, 'w18': 0.9825047792505454, 'w19': 0.42999776667485945, 'threshold': 0.32}


[I 2023-06-26 14:54:44,781] Trial 1499 finished with value: 0.6835065484046936 and parameters: {'w1': 0.1478772951635454, 'w2': 0.5503930847866912, 'w3': 0.792127290915826, 'w4': 0.8577514663418928, 'w5': 0.515115531585837, 'w6': 0.41455307154710175, 'w7': 0.15494802271667985, 'w8': 0.8187437188527878, 'w9': 0.8273743650444326, 'w10': 0.502477084457405, 'w11': 0.4948286169951065, 'w12': 0.4611466901258584, 'w13': 0.3506214269219054, 'w14': 0.7399198341252818, 'w15': 0.2874128828725182, 'w16': 0.31401520420020734, 'w17': 0.22181569338055987, 'w18': 0.9557634897689581, 'w19': 0.4905653166905885}. Best is trial 1014 with value: 0.6852377653121948.


{'w1': 0.1478772951635454, 'w2': 0.5503930847866912, 'w3': 0.792127290915826, 'w4': 0.8577514663418928, 'w5': 0.515115531585837, 'w6': 0.41455307154710175, 'w7': 0.15494802271667985, 'w8': 0.8187437188527878, 'w9': 0.8273743650444326, 'w10': 0.502477084457405, 'w11': 0.4948286169951065, 'w12': 0.4611466901258584, 'w13': 0.3506214269219054, 'w14': 0.7399198341252818, 'w15': 0.2874128828725182, 'w16': 0.31401520420020734, 'w17': 0.22181569338055987, 'w18': 0.9557634897689581, 'w19': 0.4905653166905885, 'threshold': 0.27}


{'w1': 0.07252151710540095,
 'w2': 0.7978000881421462,
 'w3': 0.6048763320288897,
 'w4': 0.8628216624820672,
 'w5': 0.11315795120721248,
 'w6': 0.10338229239082149,
 'w7': 0.39539690934281035,
 'w8': 0.6928560905271421,
 'w9': 0.04141088627980627,
 'w10': 0.47747600983627786,
 'w11': 0.13571770708884073,
 'w12': 0.248340275354829,
 'w13': 0.2937046496414661,
 'w14': 0.9682456550388799,
 'w15': 0.051840117713153916,
 'w16': 0.4156873342439348,
 'w17': 0.2435051309327308,
 'w18': 0.9488253850372468,
 'w19': 0.4902893125839418}

In [26]:
# 6, 9, 13
# 0.6854285001754761

# 0.07  | 0.6867978572845459

# {'w1': 0.016317844357604507,
#  'w2': 0.6005070573463515,
#  'w3': 0.8884213063888375,
#  'w4': 0.8445783880445076,
#  'w5': 0.7824862548087729,
#  'w6': 0.3806519186701453,
#  'w7': 0.9035265302704656,
#  'w8': 0.08253718959316077,
#  'w9': 0.0004444689861428028,
#  'w10': 0.7504741339256384,
#  'w11': 0.09067268463097893,
#  'w12': 0.4023912301444379,
#  'w13': 0.633424864342325,
#  'w14': 0.3843453836216373,
#  'w15': 0.3728444352239571,
#  'w16': 0.6751700901471811,
#  'w17': 0.8749158303582384,
#  'w18': 0.8134873047533169,
#  'w19': 0.7017477761116374}



# 0.06 | 0.6852949857711792
# {'w1': 0.05677570626120725,
#  'w2': 0.32912941583958427,
#  'w3': 0.7888415507883061,
#  'w4': 0.2699208875955819,
#  'w5': 0.68686827622187,
#  'w6': 0.22082314427422578,
#  'w7': 0.8455848214484524,
#  'w8': 0.17961926443823645,
#  'w9': 0.15386702893860843}


# [I 2023-06-16 19:56:17,542] | TH:0.11 | Trial 2772 finished with value: 0.685997724533081 and parameters: {'w1': 0.2728695549458137, 'w2': 0.32228366602907965, 'w3': 0.5198602860436105, 'w4': 0.3645111903670712, 'w5': 0.9222454111482654, 'w6': 0.513409920082969, 'w7': 0.8675021234700891, 'w8': 0.0722146925058941, 'w9': 0.09988548514924554, 'w10': 0.04023375737773327}. Best is trial 2772 with value: 0.685997724533081.

#  th:0.10 | Best is trial 665 with value: 0.686022162437439.
# {'w1': 0.07249514832914417,
#  'w2': 0.3044527524365949,
#  'w3': 0.46339624897419424,
#  'w4': 0.30916961792390085,
#  'w5': 0.9820263596044505,
#  'w6': 0.13629275856955803,
#  'w7': 0.9819243920033219,
#  'w8': 0.10786742006569568,
#  'w9': 0.017963370847088496,
#  'w10': 0.2269773365337288}




# th:0.09 Trial 599 finished with value: 0.6861019730567932 and parameters: {'w1': 0.08411983424429813, 'w2': 0.2397144985551996, 'w3': 0.565597681400598, 'w4': 0.27365134829998194, 'w5': 0.8519007002554051, 'w6': 0.0034467714906157854, 'w7': 0.9632070992690323, 'w8': 0.036223969967080535, 'w9': 0.1408041912398369, 'w10': 0.0808046168343181}. Best is trial 599 with value: 0.6861019730567932.






# Best is trial 979 th:0.08 | with value: 0.6860898733139038.
# {'w1': 0.04219070925378117,
#  'w2': 0.2814610861485835,
#  'w3': 0.45306132583945036,
#  'w4': 0.08300507686599938,
#  'w5': 0.9689922153717234,
#  'w6': 0.5832699926479489,
#  'w7': 0.8380073187640669,
#  'w8': 0.00027139503974531664,
#  'w9': 0.08843006287891712,
#  'w10': 0.2639847795838674}




# 0.6835659742355347 and parameters: {'w1': 0.012511663696378001, 'w2': 0.7308678705489249, 'w3': 0.7309363859834449, 'w4': 0.8998292052691779, 'w5': 0.9377896156028775}. Best is trial 183 with value: 0.6835659742355347.
# TH: 0.1

# pl3_round2_s2 included at last
# Trial 177 finished with value: 0.6850342154502869 and parameters: 
#{'w1': 0.1976000599839957, 'w2': 0.4126970697935236, 'w3': 0.0010400328806771247, 'w4': 0.13358923329787326, 'w5': 0.6293367385421876, 'w6': 0.8157294847307331}. Best is trial 1286 with value: 0.6850342154502869.
#TH:0.19


# 0.6850342154502869
# {'w1': 0.1976000599839957,
#  'w2': 0.4126970697935236,
#  'w3': 0.0010400328806771247,
#  'w4': 0.13358923329787326,
#  'w5': 0.6293367385421876,
#  'w6': 0.8157294847307331}



#  0.6864011883735657 and parameters: 
# {'w1': 0.0037528376744131176, 'w2': 0.1968372043477828, 'w3': 0.324173594919189,
#  'w4': 0.08069037545373889, 'w5': 0.666119680919894, 'w6': 0.23792233503063126, 'w7': 0.6474738231385013, 
#  }
#. Best is trial 301 with value: 0.6864011883735657.
# TH: 0.11



#th: 0.2
# {'w1': 0.9118427143017789, 'w2': 0.07722806128805348, 'w3': 0.3483576258941602}


# 0.6802493929862976
# {'w1': 0.901456237984701,
#  'w2': 0.03333500127663481,
#  'w3': 0.1012744451767772,
#  'w4': 0.3315684160625839}


# 0.6802645325660706
# {'w1': 0.9423043059931125,
#  'w2': 0.12999481950972483,
#  'w3': 0.34096541343299724}




# 0.6753096580505371
# {'w1': 0.8191544364616984,
#  'w2': 0.2172873170900641,
#  'w3': 0.9872223348463418,
#  'w4': 0.34763898831692425,
#  'w5': 0.6106201134139329}
